<a href="https://colab.research.google.com/github/chihpoc/chihpoc/blob/main/Polars_and_DuckDB_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet -U 'polars[pyarrow]'
!pip install --quiet duckdb

     |████████████████████████████████| 13.2 MB 32.3 MB/s 
     |████████████████████████████████| 16.4 MB 9.4 MB/s 


In [ ]:
import duckdb
import polars as pl
import pandas as pd

## Pandas to Polars to Arrow to DuckDB

In [ ]:
df = pd.DataFrame(
    {
        "A": [1, 2, 3, 4, 5],
        "fruits": ["banana", "banana", "apple", "apple", "banana"],
        "B": [5, 4, 3, 2, 1],
        "cars": ["beetle", "audi", "beetle", "beetle", "beetle"],
    }
)
df

,A,fruits,B,cars
0,1,banana,5,beetle
1,2,banana,4,audi
2,3,apple,3,beetle
3,4,apple,2,beetle
4,5,banana,1,beetle


In [ ]:
polars_df = pl.DataFrame(df)
polars_df

A,fruits,B,cars
i64,str,i64,str
1,"""banana""",5,"""beetle"""
2,"""banana""",4,"""audi"""
3,"""apple""",3,"""beetle"""
4,"""apple""",2,"""beetle"""
5,"""banana""",1,"""beetle"""


In [ ]:
polars_to_arrow = (
    polars_df
    .sort("fruits")
    .select(
        [
            "fruits",
            "cars",
            pl.lit("fruits").alias("literal_string_fruits"),
            pl.col("B").filter(pl.col("cars") == "beetle").sum(),
            pl.col("A").filter(pl.col("B") > 2).sum().over("cars").alias("sum_A_by_cars"),     # groups by "cars"
            pl.col("A").sum().over("fruits").alias("sum_A_by_fruits"),                         # groups by "fruits"
            pl.col("A").reverse().over("fruits").alias("rev_A_by_fruits"),                     # groups by "fruits
            pl.col("A").sort_by("B").over("fruits").alias("sort_A_by_B_by_fruits"),            # groups by "fruits"
        ]
    )
    .to_arrow()
)
polars_to_arrow

pyarrow.Table
fruits: large_string
cars: large_string
literal_string_fruits: large_string
B: int64
sum_A_by_cars: int64
sum_A_by_fruits: int64
rev_A_by_fruits: int64
sort_A_by_B_by_fruits: int64
----
fruits: [["apple","apple","banana","banana","banana"]]
cars: [["beetle","beetle","beetle","audi","beetle"]]
literal_string_fruits: [["fruits","fruits","fruits","fruits","fruits"]]
B: [[11,11,11,11,11]]
sum_A_by_cars: [[4,4,4,2,4]]
sum_A_by_fruits: [[7,7,8,8,8]]
rev_A_by_fruits: [[4,3,5,2,1]]
sort_A_by_B_by_fruits: [[4,3,5,2,1]]

In [ ]:
output = duckdb.query("""
  SELECT 
    fruits,
    first(sum_A_by_fruits) as sum_A
  FROM polars_to_arrow
  GROUP BY ALL
  ORDER BY ALL
""").arrow()
output

pyarrow.Table
fruits: string
sum_A: int64
----
fruits: [["apple","banana"]]
sum_A: [[7,8]]

## Pandas to DuckDB to Arrow to Polars

In [ ]:
duckdb_to_arrow = duckdb.query("""
  SELECT
    fruits,
    cars,
    'fruits' as literal_string_fruits,
    SUM(B) FILTER (cars = 'beetle') OVER () as B,
    SUM(A) FILTER (B > 2) OVER (PARTITION BY cars) as sum_A_by_cars,
    SUM(A) OVER (PARTITION BY fruits) as sum_A_by_fruits
  FROM df
  ORDER BY
    fruits,
    df.B
""").arrow()
duckdb_to_arrow

pyarrow.Table
fruits: string
cars: string
literal_string_fruits: string
B: decimal128(38, 0)
sum_A_by_cars: decimal128(38, 0)
sum_A_by_fruits: decimal128(38, 0)
----
fruits: [["apple","apple","banana","banana","banana"]]
cars: [["beetle","beetle","beetle","audi","beetle"]]
literal_string_fruits: [["fruits","fruits","fruits","fruits","fruits"]]
B: [[11,11,11,11,11]]
sum_A_by_cars: [[4,4,4,2,4]]
sum_A_by_fruits: [[7,7,8,8,8]]

In [ ]:
polars_df_2 = pl.DataFrame(duckdb_to_arrow)
polars_df_2

fruits,cars,literal_string_fruits,B,sum_A_by_cars,sum_A_by_fruits
str,str,str,f64,f64,f64
"""apple""","""beetle""","""fruits""",11.0,4.0,7.0
"""apple""","""beetle""","""fruits""",11.0,4.0,7.0
"""banana""","""beetle""","""fruits""",11.0,4.0,8.0
"""banana""","""audi""","""fruits""",11.0,2.0,8.0
"""banana""","""beetle""","""fruits""",11.0,4.0,8.0


In [ ]:
output_2 = (
    polars_df_2
    .groupby('fruits')
    .agg(
        pl.col('sum_A_by_fruits')
        .first()
        .sort_by('fruits')
        )
).to_arrow()
output_2

pyarrow.Table
fruits: large_string
sum_A_by_fruits: double
----
fruits: [["apple","banana"]]
sum_A_by_fruits: [[7,8]]